## Calculating optimal leverage using the Kelly Formula

The Kelly formula calculates the optimal capital allocation and leverage to apply to each strategy in a portfolio of strategies so as to maximize the long term growth rate of the portfolio.

Kelly should be used to denote the maximum leverage allowed for a strategy, not the total to use in one order (especially for higher frequency strategies).

Notes:
- Kelly betting allows insurance against total ruin (prevents total loss of capital)
- Kelly betting requires continuous reallocation of the portfolio based on changes in strategy return means and standard deviations ($\geq$ 1x per day)
- Lookback period for calculation of $F^*$ should be a function of the average holding period of each strategy, or 6 months for intraday only strategies
- Assume that all trading profits are reinvested into the portfolio
- The method is slightly flawed as we will approximate the returns (excess returns) distribution of each strategy to be Gaussian

Formula for optimal capital allocation:

#### $F^{*} = C^{-1}M$
- $F^{*}$: optimal fractions of total equity that should be allocated to each strategy $f_{1...N}$
- $C$: covariance matrix where $C_{i, j} is the covariance of the returns between strategies i and j (-1 because inverse)
- $M$: column vector of the mean returns of each strategy (uncompounded, unlevered returns)


Assuming all strategies are independent of eachother, the covariance matrix becomes diagonal and we can simplify the above to:

#### Kelly Formula: $f_{i} = \frac{m_{i}}{s_{i}^2}$
- $m_{i}$: $i^{th}$ strategy's mean returns
- $s_{i}$: $i^{th}$ strategy's standard deviation of returns

Using Kelly to calculate the optimal leverage allocation between a portfolio of three ETFs

In [2]:
# imports
import pandas as pd
import numpy as np
from numpy.linalg import inv

In [3]:
# Data ingestion and EDA
oih_df = pd.read_excel('OIH.xls')
rkh_df = pd.read_excel('RKH.xls')
df = pd.merge(oih_df, rkh_df, on='Date', suffixes=('_OIH', '_RKH'))
rth_df = pd.read_excel('RTH.xls')
df = pd.merge(df, rth_df, on='Date')
df.rename(columns={'Adj Close': 'Adj Close_RTH'}, inplace=True)
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df.head()

,Open_OIH,High_OIH,Low_OIH,Close_OIH,Volume_OIH,Adj Close_OIH,Open_RKH,High_RKH,Low_RKH,Close_RKH,Volume_RKH,Adj Close_RKH,Open,High,Low,Close,Volume,Adj Close_RTH
Date,,,,,,,,,,,,,,,,,,
2001-05-17,91.25,94.00,90.25,93.93,276900,84.60,115.40,117.00,115.40,116.25,18500,93.63,97.75,98.45,97.70,98.00,14200,87.09
2001-05-18,94.00,96.04,93.80,94.70,546300,85.30,116.50,116.70,115.85,115.85,81800,93.31,97.30,97.75,97.25,97.75,1900,86.87
2001-05-21,95.10,95.58,93.96,95.55,627500,86.06,116.00,116.00,115.00,116.00,26500,93.43,98.20,100.15,97.80,99.95,15900,88.83
2001-05-22,95.46,95.46,93.70,94.10,371700,84.76,115.80,118.60,115.80,118.13,37900,95.15,99.75,100.90,99.15,100.10,88500,88.96
2001-05-23,93.00,93.00,90.52,90.57,682400,81.58,117.75,118.16,117.41,118.00,21300,95.04,98.80,98.80,97.95,98.20,4300,87.27


In [4]:
# Calculate daily returns
daily_ret = df.loc[:, ('Adj Close_OIH', 'Adj Close_RKH', 'Adj Close_RTH')].pct_change()
daily_ret.rename(columns={'Adj Close_OIH': 'OIH', 'Adj Close_RKH': 'RKH', 'Adj Close_RTH': 'RTH'}, inplace=True)
rfr = 0.04
excess_ret = daily_ret - rfr/252
excess_ret.head()

,OIH,RKH,RTH
Date,,,
2001-05-17,NaN,NaN,NaN
2001-05-18,0.008116,-0.003576,-0.002685
2001-05-21,0.008751,0.001127,0.022404
2001-05-22,-0.015264,0.018251,0.001305
2001-05-23,-0.037676,-0.001315,-0.019156


In [6]:
M = 252 * excess_ret.mean()  # The mean of the excess returns
M

OIH    0.139568
RKH    0.029400
RTH   -0.007346
dtype: float64

In [7]:
C = 252 * excess_ret.cov()  # The covariance matrix of the excess returns
C

,OIH,RKH,RTH
OIH,0.110901,0.020014,0.018255
RKH,0.020014,0.037165,0.026893
RTH,0.018255,0.026893,0.041967


In [8]:
F = np.dot(inv(C), M)  # The factor loadings
F

array([ 1.2919082 ,  1.17226473, -1.48821285])

In [14]:
g = .04 + np.dot(F.T, np.dot(C, F))/2  # Max CAGR
print(f'Maximum annualized compounted growth rate: {g:.4f}')

Maximum annualized compounted growth rate: 0.1529


In [15]:
S = np.sqrt(np.dot(F.T, np.dot(C, F)))  # Portfolio Sharpe
print(f'Sharpe Ratio of Portfolio: {S:.4f}')

Sharpe Ratio of Portfolio: 0.4751


## Graduated Kelly betting for extra risk control

Rather than using the optimal leverage suggested by the Kelly formula, we can use a combination of "half-kelly betting" and maximum tolerable capital loss to allocate capital.

Revised Formula:

$leverage = min(\frac{T_p}{D_p}, \frac{kelly}{2})$
- $T_p$: maximum tolerable equity loss for one period $p$
- $D_p$: maximum drawdown recorded in one period $p$